# 알파 채널이 추가된 png 파일 만들기
* jpg 이미지를 불러와 각 기능들을 수행하여 선택할 영역을 추출한 후, 알파채널을 추가해 png 파일로 저장하는 프로그램


### 프로그램 실행 방법
1. 편집하려는 이미지의 경로를 직접 입력 : 옳은 경로를 입력할 때까지 반복
2. 실행할 기능에 대한 버튼을 키보드로부터 입력 : [r,1,2,3,i,+,-,q,p,s] 중 입력
    * 'r': roi 영역 선택
    * '1': 역투영
        * 드래그 한 영역에 대한 역투영
    * '2': 흑백 이미지 쓰레시 홀드
        * 't': 선택영역 반전 토글
        * 'm': 적응형/전역 쓰레시홀드 토글
        * 트랙바로 값을 조절하여 이진화
    * '3': hsv 마스킹
        * 클릭 좌표 픽셀로 마스킹
        * 트랙바로 조절하여 마스킹
    * 'i': 이미지 초기화
    * '+': 이미지 확대
    * '-': 이미지 축소
    * 'q': 종료
    * 'p': 미리보기
    * 's': 저장
3. (1,2,3) 기능으로 추출한 이미지에 대해 키보드 입력 : [s,d,x,c,q,ESC] 중 입력
    * 's': 저장할 이미지에 선택 영역 더한 후 모폴로지 열림 연산(침식, 팽창) 수행
    * 'd': 저장할 이미지서 선택 영역 뺀 후 모폴로지 닫기 연산(팽창, 침식)을 하는 기능
    * 'x': 저장할 이미지에 선택 영역을 더함
    * 'c': 저장할 이미지서 선택 영역을 뺌
    * ESC, 'q': 종료
    
    
    
* 자세한 사항은 사용자 매뉴얼 '구성도' 참조

In [4]:
## 윈도우 창 사이즈 조절 함수
def reset_window_size(window_name, img):
    if img.shape[1] < 400:
        cv2.resizeWindow(window_name, 400, img.shape[0])


# 이미지 초기화 함수
def init_img_and_mask():
    global img_copy, img_binary_mask
    img_copy = img.copy()
    img_binary_mask = np.zeros((img_copy.shape[0], img_copy.shape[1]), np.uint8)


def resize_img(ratio, param):
    global img_copy, img_binary_mask
    img_copy = cv2.resize(img_copy, None, None, ratio, ratio, interpolation = param)
    img_binary_mask = np.zeros((img_copy.shape[0], img_copy.shape[1]), np.uint8)
    

# roi를 선택해 반환하는 함수
def roi_copy(window_name, img_from):
    x,y,w,h = cv2.selectROI(window_name, img_from, False) # selectROI OpenCV 함수
    cv2.destroyWindow(window_name) # 창 닫기
    if w and h: # 값이 존재하면
        roi = img_from[y:y+h, x:x+w] # roi 영역 referencing
        return roi
    else: # 아무것도 읽어온게 없으면
        return -1


# 역투영 기능을 수행하는 함수
def back_project(window_name, img_copy):
    img_hsv = cv2.cvtColor(img_copy, cv2.COLOR_BGR2HSV) # 이미지를 hsv 형식으로 변환
    roi = roi_copy(window_name, img_copy) # 역투영을 위한 roi 추출
    if type(roi) != int: # 반환값이 int형이 아닌 경우
        roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV) # roi를 hsv 형식으로 변환
        roi_hist = cv2.calcHist([roi_hsv],[0, 1], None, [180, 256], [0, 180, 0, 256] ) # hue값, saturation값에 대한 히스토그램 계산

        img_backproject = cv2.calcBackProject([img_hsv], [0, 1], roi_hist,  [0, 180, 0, 256], 1) # OpenCV 역투영 함수 호출하여 결과를 저장

        kernal = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)) # 5x5 크기의 타원형 커널 생성, # 표면을 부드럽게 해주는 코드
        cv2.filter2D(img_backproject, -1, kernal, img_backproject) # 원본과 동일한 dtype으로 kernal에 대한 블러링
        
        _, mask_backproject = cv2.threshold(img_backproject, 1, 255, cv2.THRESH_BINARY) # 쓰레시 홀딩

        while True: # 무한루프
            preview(img_copy, mask_backproject) # 미리보기
            key = cv2.waitKey(0)
            if key == 27 or key == 113:
                cv2.destroyAllWindows()
                break
            elif key == ord("s"):
                save_to_mask(mask_backproject, 0)
            elif key == ord("d"):
                save_to_mask(mask_backproject, 1)
            elif key == ord("x"):
                save_to_mask(mask_backproject, 2)
            elif key == ord("c"):
                save_to_mask(mask_backproject, 3)


# 받아온 마스크와 겹치는 이미지 미리보기 출력
def preview(img_copy, img_mask):
    img_result = cv2.bitwise_and(img_copy, img_copy, mask=img_mask) # bitwise 연산으로 마스크에 해당하는 좌표값만 추출
    cv2.imshow("preview", img_result)


# 읽어온 마스크를 img_binary_mask에 더하거나 빼는 함수
def save_to_mask(img_from, flag):
    global img_binary_mask
    kernal = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)) # 5,5 크기의 타원형 커널 생성
    if flag in [0,2]: # 마스크를 더하고
        img_binary_mask = cv2.add(img_binary_mask, img_from)
        if flag == 0: # 닫기 연산(팽창, 침식) 수행하는 기능
            img_binary_mask = cv2.morphologyEx(img_binary_mask, cv2.MORPH_CLOSE, kernal)
        
    elif flag in [1,3]: # 마스크를 빼고
        img_binary_mask = cv2.subtract(img_binary_mask, img_from)
        if flag == 1: # 열림 연산(침식, 팽창) 수행하는 기능
            img_binary_mask = cv2.morphologyEx(img_binary_mask, cv2.MORPH_OPEN, kernal)
    
    cv2.imshow("preview", img_binary_mask)
    cv2.waitKey(0)
    cv2.destroyWindow("preview")


# 이미지를 흑백으로 변환해 이진화 시키는 기능을 수행하는 함수
def binary_threshold(window_name, img_copy):
    img_gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY) # 이미지 흑백으로 변환
    flag = cv2.THRESH_BINARY                               # 초기 flag 설정

    blockSize = 3       # 블럭 사이즈 초기값 (blockSize % 2 == 1 && blockSize > 1 이어야 함)
    C = 0               # 차감 상수 초기값
    threshold = True   # 쓰레시홀드 전역 / 적응형 선택 변수
    
    cv2.namedWindow(window_name) # 창 이름 설정
    cv2.createTrackbar('value', window_name, 0, 255, for_threshold_trackbar) # 0~255 값 트랙바 생성
    cv2.setTrackbarPos('value', window_name, 127)   # 트랙바 초기값 127 설정

    # 쓰레시홀드 연산 마칠 때 까지 무한 반복
    while True:
        if threshold: # 전역 쓰레시홀드
            value = cv2.getTrackbarPos('value', window_name)   # 임계값 받아오기
            _, img_binary = cv2.threshold(img_gray, value, 255, flag) # 쓰레시 홀드 
            cv2.imshow(window_name, img_binary)
            reset_window_size(window_name, img_binary)

        else:         # 적응형 쓰레시홀드
            size = cv2.getTrackbarPos('blockSize', window_name+'2')  # 블럭 사이즈
            if size%2 and size>1:   # blockSize % 2 == 1 && blockSize > 1 조건 불충족 시 오류 발생
                blockSize = size
            C = cv2.getTrackbarPos('C', window_name+'2')             # 차감 상수 

            img_binary = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, flag, blockSize, C) # 적응형
            cv2.imshow(window_name+'2', img_binary)
            reset_window_size(window_name+'2', img_binary)

        key = cv2.waitKey(1)&0xFF # non-block 상태로, 입력이 있는지 확인해서 있으면 아래 조건문 검사
        if key != 255: # 입력이 있을 경우에만 (불필요한 조건 검사 줄이기)
            if key == 27 or key == 113: # 닫기
                cv2.destroyAllWindows()
                break
            elif key == ord("s"):
                save_to_mask(img_binary, 0) # 더해서 저장, 닫힘 연산
            elif key == ord("d"):
                save_to_mask(img_binary, 1) # 빼서 저장, 열림 연산
            elif key == ord("x"):
                save_to_mask(img_binary, 2) # 더해서 저장
            elif key == ord("c"):
                save_to_mask(img_binary, 3) # 빼서 저장
                
            elif key == ord('t'): # t - 쓰레시홀드 결과값 반전 토글형식으로 선택
                if flag == cv2.THRESH_BINARY:
                    flag = cv2.THRESH_BINARY_INV # THRESH_BINARY_INV : 반전된 마스크 이미지
                else:
                    flag = cv2.THRESH_BINARY # cv2.THRESH_BINARY: 임계값 이상은 value로, 미만은 0으로 지정
            elif key == ord('m'): # m - 쓰레시홀드 전역 / 적응형 토글형식으로 선택
                if threshold:
                    threshold = False
                    cv2.destroyWindow(window_name)
                    cv2.namedWindow(window_name+'2') # 창 이름 설정
                    cv2.createTrackbar('blockSize', window_name+'2', 0, 255, for_threshold_trackbar) # 트랙바 생성
                    cv2.createTrackbar('C', window_name+'2', 0, 80, for_threshold_trackbar) # 트랙바 생성
                    cv2.setTrackbarPos('blockSize', window_name+'2', 0) # 트랙바 초기값 설정
                    cv2.setTrackbarPos('C', window_name+'2', 0) # 트랙바 초기값 설정
                else:
                    threshold = True
                    cv2.destroyWindow(window_name+'2')
                    cv2.namedWindow(window_name) # 창 이름 설정
                    cv2.createTrackbar('value', window_name, 0, 255, for_threshold_trackbar) # 트랙바 생성
                    cv2.setTrackbarPos('value', window_name, 127) # 트랙바 초기값 설정


def hsv_masking(window_name, img_copy):
    img_hsv = cv2.cvtColor(img_copy, cv2.COLOR_BGR2HSV) # BGR에서 HSV형식으로 변환
    hsv = 0 # hue, saturation, value 값
    lower_hsv = 0 # 범위 최소
    upper_hsv = 0 # 범위 최대
    offset = 20 # default offset

    cv2.namedWindow(window_name) # 값을 뽑아낼 영상의 창 이름
    cv2.namedWindow('img_result') # 각 값으로 뽑아낸 결과 출력창 이름
    cv2.setMouseCallback(window_name, mouse_callback_masking) # 마우스 콜백 함수 등록

    cv2.createTrackbar('Hue', 'img_result', 0, 180, for_hsv_trackbar)
    cv2.createTrackbar('Saturation', 'img_result', 0, 255, for_hsv_trackbar)
    cv2.createTrackbar('Value', 'img_result', 0, 255, for_hsv_trackbar)
    cv2.createTrackbar('Offset', 'img_result', 0, 100, for_hsv_trackbar)
    cv2.setTrackbarPos('Hue', 'img_result', 0)
    cv2.setTrackbarPos('Saturation', 'img_result', 0)
    cv2.setTrackbarPos('Value', 'img_result', 0)
    cv2.setTrackbarPos('Offset', 'img_result', 20)

    while True:
        hue = cv2.getTrackbarPos('Hue', 'img_result')
        saturation = cv2.getTrackbarPos('Saturation', 'img_result')
        value = cv2.getTrackbarPos('Value', 'img_result')
        offset = cv2.getTrackbarPos('Offset', 'img_result')

        # hue값과 유사한 픽셀값의 범위 설정, hue값 +- offset
        lower_hsv = np.array([hue-offset,saturation-offset,value-offset])
        upper_hsv = np.array([hue+offset,saturation+offset,value+offset])


        # 유사한 범위 안에 해당하는 값은 그대로, 아닌 부분은 0으로 바꾸어 반환
        img_mask = cv2.inRange(img_hsv, lower_hsv, upper_hsv)

        # img_copy 에 대하여 img_mask 각 요소 값이 0이 아닌 픽셀들만 골라내는 연산
        img_result = cv2.bitwise_and(img_copy, img_copy, mask=img_mask)

        cv2.imshow(window_name, img_copy)
        cv2.imshow('img_mask', img_mask)
        cv2.imshow('img_result', img_result)
        reset_window_size('img_result', img_result)

        key = cv2.waitKey(1)
        if key != 255:
            if key == 27 or key == 113: # 닫기
                cv2.destroyAllWindows()
                break
            elif key == ord("s"):
                save_to_mask(img_mask, 0) # 더해서 저장, 닫힘 연산
            elif key == ord("d"):
                save_to_mask(img_mask, 1) # 빼서 저장, 열림 연산
            elif key == ord("x"):
                save_to_mask(img_mask, 2) # 더해서 저장
            elif key == ord("c"):
                save_to_mask(img_mask, 3) # 빼서 저장
    
    
#  마스킹 창에서 마우스 클릭시 호출되는 함수
def mouse_callback_masking(event, x, y, flags, param): 
    # 왼쪽 클릭 했을 때
    if event == cv2.EVENT_LBUTTONDOWN:
        # 해당 좌표의 픽셀값(bgr)을 받아와서, shape[3]을 [1][1][3]형식으로 변환하여 저장
        pixel = np.uint8([[img_copy[y,x]]]) 
        hsv = cv2.cvtColor(pixel, cv2.COLOR_BGR2HSV) # BGR 형식을 HSV 형식으로 변환
        
        # 선택한 좌표의 hsv 값들로 트랙바의 값을 셋팅
        cv2.setTrackbarPos('Hue', 'img_result', hsv[0][0][0]) # 색상(Hue) 세팅
        cv2.setTrackbarPos('Saturation', 'img_result', hsv[0][0][1]) # 채도(Saturation) 셋팅
        cv2.setTrackbarPos('Value', 'img_result', hsv[0][0][2]) # 밝기(Value) 셋팅    
        
        
# 쓰레시홀드 트랙바 생성을 위한 콜백함수
def for_threshold_trackbar(x):
    pass # 아무일도 수행하지 않음 - 반복문에서 수행


# hsv 값에 대한 트랙바 생성을 위한 콜백함수
def for_hsv_trackbar(x):
    pass # 아무일도 수행하지 않음 - 반복문에서 수행


######################## main 부분 ########################
import cv2
import numpy as np

# 해당 경로에 이미지가 존재할 때 까지 반복
while True:
    img_file = input("이미지 경로 :") #'img/t3.jpg' #
    img = cv2.imread(img_file)
    if img is None:
        print("경로에 이미지가 존재하지 않습니다.")
        continue
    init_img_and_mask()
    break

# 프로세스 수행 반복, 키보드 입력으로 다음 기능을 선택
while True:
    print("r: roi 영역 선택\n1: 역투영\n2: 쓰레시 홀드\n3: hsv 마스킹")
    print("+: 이미지 확대\n-: 이미지 축소\ni: 이미지 초기화\np: 미리보기\ns: 저장\nq: 종료\n")
    cv2.imshow('Current IMG', img_copy)
    
    # 키보드 입력으로 작업 선택 [esc,+,-,1,2,3,i,p,q,r,s]
    while True:
        key = cv2.waitKey(0) & 0xFF
        # ascii code {esc,+,-,1,2,3,i,p,q,r,s}
        if key in {27,43,45,49,50,51,105,112,113,114,115}: 
            cv2.destroyWindow('Current IMG')
            break
    
    # q - 닫기
    if key == ord('q'): # to ascii code
        break
        
    # + - 그림 확대
    elif key == ord('+'): # 배율 지정으로 확대, CUBIC
        resize_img(1.2, cv2.INTER_CUBIC)
        
    # - - 그림 축소
    elif key == ord('-'): # 크기 지정으로 축소, AREA
        resize_img(0.8, cv2.INTER_AREA)
        
    # i - Current IMG 초기 이미지로 초기화
    elif key == ord('i'):
        init_img_and_mask()

    # p - 저장될 이미지 미리보기
    elif key == ord('p'):
        preview(img_copy, img_binary_mask)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    # s - 저장
    elif key == ord('s'):
        cv2.destroyWindow('Current IMG')
        
        # 알파 채널이 존재하는 BGRA 형식으로 변환
        img_result = cv2.cvtColor(img_copy.copy(), cv2.COLOR_BGR2BGRA)
        img_result[:, :, 3] = 0 # 알파채널을 전부 0으로 초기화
        img_result[:, :, 3] += img_binary_mask # 알파채널을 전부 mask 값으로 변경
        cv2.imwrite(img_file[:-3]+"png", img_result) # 파일경로이름.png로 저장
        
    # r - roi 영역 선택
    elif key == ord('r'):
        result = roi_copy('ROI Select', img_copy)
        if type(result) != int: # 아무것도 roi를 아무것도 잡지 않은 경우, -1(int)을 반환함
            img_copy = result
            img_binary_mask = np.zeros((img_copy.shape[0], img_copy.shape[1]), np.uint8)
    
    # 1번 - 역투영 방식 선택했을 때
    elif key == ord('1'):
        back_project('Back Project', img_copy)
        
    # 2번 - 흑백 사진에 대한 쓰레시 홀드를 통한 이분화작업 수행
    elif key == ord('2'):
        binary_threshold('Binary Threshold', img_copy)
    
    # 3번 - hsv 마스킹 방식 눌렀을 때
    elif key == ord('3'):
        hsv_masking('hsv_masking', img_copy)
                
print("종료합니다.")
# 이미지 경로 예시 : img/t3.jpg

이미지 경로 :img/t3.jpg
r: roi 영역 선택
1: 역투영
2: 쓰레시 홀드
3: hsv 마스킹
+: 이미지 확대
-: 이미지 축소
i: 이미지 초기화
p: 미리보기
s: 저장
q: 종료

r: roi 영역 선택
1: 역투영
2: 쓰레시 홀드
3: hsv 마스킹
+: 이미지 확대
-: 이미지 축소
i: 이미지 초기화
p: 미리보기
s: 저장
q: 종료

r: roi 영역 선택
1: 역투영
2: 쓰레시 홀드
3: hsv 마스킹
+: 이미지 확대
-: 이미지 축소
i: 이미지 초기화
p: 미리보기
s: 저장
q: 종료

r: roi 영역 선택
1: 역투영
2: 쓰레시 홀드
3: hsv 마스킹
+: 이미지 확대
-: 이미지 축소
i: 이미지 초기화
p: 미리보기
s: 저장
q: 종료

r: roi 영역 선택
1: 역투영
2: 쓰레시 홀드
3: hsv 마스킹
+: 이미지 확대
-: 이미지 축소
i: 이미지 초기화
p: 미리보기
s: 저장
q: 종료

r: roi 영역 선택
1: 역투영
2: 쓰레시 홀드
3: hsv 마스킹
+: 이미지 확대
-: 이미지 축소
i: 이미지 초기화
p: 미리보기
s: 저장
q: 종료

r: roi 영역 선택
1: 역투영
2: 쓰레시 홀드
3: hsv 마스킹
+: 이미지 확대
-: 이미지 축소
i: 이미지 초기화
p: 미리보기
s: 저장
q: 종료

r: roi 영역 선택
1: 역투영
2: 쓰레시 홀드
3: hsv 마스킹
+: 이미지 확대
-: 이미지 축소
i: 이미지 초기화
p: 미리보기
s: 저장
q: 종료

r: roi 영역 선택
1: 역투영
2: 쓰레시 홀드
3: hsv 마스킹
+: 이미지 확대
-: 이미지 축소
i: 이미지 초기화
p: 미리보기
s: 저장
q: 종료

r: roi 영역 선택
1: 역투영
2: 쓰레시 홀드
3: hsv 마스킹
+: 이미지 확대
-: 이미지 축소
i: 이미지 초기화
p: 미리보기
s: 저장
q: 종료

r: roi 영역 선택
1: 역투영
2: 쓰레시 홀드
3: hsv 마스킹
+: 이미지 확대


# 빈 이미지 배열 저장하기
* 알파채널을 포함하는 900\*900 크기의 빈 이미지 배열을 만들어 저장하는 프로그램

In [ ]:
import cv2
import numpy as np

img = np.full((900,900,4), 255, dtype=np.uint8)
img[:, :, 3] = 0
cv2.imshow('test', img)
cv2.imwrite('img/blank_900.png', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 추출한 png 이미지를 추가하는 코디 시뮬레이터

### 프로그램 실행 방법

1. 편집하려는 이미지의 경로를 직접 입력 : 옳은 경로를 입력할 때까지 반복
2. 실행할 기능에 대한 버튼을 키보드 또는 마우스 입력 : [우클릭, 좌클릭,i,s,q] 중 입력
    * 우클릭 : 좌표에 이미지 미리 출력하기 (떼면 사라짐)
    * 좌클릭 : 좌표에 이미지 추가하기
    * 'i': 이미지 초기화
    * 's': 저장
    * 'q': 종료

In [1]:
######################## 함수 define 부분 ########################

def mouse_callback_cody(event, x, y, flags, param):
    global img_clothes, img_before, img_add, not_alpha_area, w, h
    img_roi = img_add.copy()
    y -= h//2
    x -= w//2
    min_x = 0
    min_y = 0
    max_y = img_clothes.shape[0]
    max_x = img_clothes.shape[1]
    
    # 우클릭 눌렸을 떄
    if event == cv2.EVENT_RBUTTONDOWN:
        img_before = img_clothes.copy() # 다시 되돌아가기 위해 필요한 임시 저장 변수
        
        # 오른쪽 위로 넘어갔을 때
        if x+w> max_x and y < 0:
            img_roi = img_roi[(-y):, :-(x+w-max_x)]
            max_x -= x
            min_y += -y
            img_clothes[:y+h,x:x+w] = cv2.bitwise_and(img_clothes[:y+h,x:x+w], img_clothes[:y+h,x:x+w],
                                                      mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[:y+h,x:x+w] = cv2.add(img_clothes[:y+h,x:x+w], img_roi)
        
        # 오른쪽 아래로 넘어갔을 때
        elif y+h > max_y and x+w > max_x:
            img_roi = img_roi[:-(y+h-max_y), :-(x+w-max_x)]
            max_y -= y
            max_x -= x
            img_clothes[y:y+h,x:x+w] = cv2.bitwise_and(img_clothes[y:y+h,x:x+w], img_clothes[y:y+h,x:x+w], 
                                                       mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,x:x+w] = cv2.add(img_clothes[y:y+h,x:x+w], img_roi)
        
        # 왼쪽 위로 넘어갔을 때
        elif y < 0 and x < 0:
            img_roi = img_roi[(-y):, (-x):]
            min_y += -y
            min_x += -x
            img_clothes[:y+h,:x+w] = cv2.bitwise_and(img_clothes[:y+h,:x+w], img_clothes[:y+h,:x+w], 
                                                     mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[:y+h,:x+w] = cv2.add(img_clothes[:y+h,:x+w], img_roi) 
            
        # 왼쪽 아래로 넘어갔을 때
        elif x < 0 and y+h > max_y:
            img_roi = img_roi[:-(y+h-max_y), (-x):]
            max_y -= y
            min_x += -x
            img_clothes[y:y+h,:x+w] = cv2.bitwise_and(img_clothes[y:y+h,:x+w], img_clothes[y:y+h,:x+w],
                                                      mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,:x+w] = cv2.add(img_clothes[y:y+h,:x+w], img_roi)
        
        # 위로 넘어갔을 때
        elif y < 0:
            img_roi = img_roi[(-y):, :]
            min_y += -y
            img_clothes[:y+h,x:x+w] = cv2.bitwise_and(img_clothes[:y+h,x:x+w], img_clothes[:y+h,x:x+w],
                                                      mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[:y+h,x:x+w] = cv2.add(img_clothes[:y+h,x:x+w], img_roi)
          
        # 아래로 넘어갔을 때
        elif y+h > max_y:
            img_roi = img_roi[:-(y+h-max_y), :]
            max_y -= y
            img_clothes[y:y+h,x:x+w] = cv2.bitwise_and(img_clothes[y:y+h,x:x+w], img_clothes[y:y+h,x:x+w],
                                                       mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,x:x+w] = cv2.add(img_clothes[y:y+h,x:x+w], img_roi)
            
        # 왼쪽으로 넘어갔을 때
        elif x < 0:
            img_roi = img_roi[:, (-x):]
            min_x += -x
            img_clothes[y:y+h,:x+w] = cv2.bitwise_and(img_clothes[y:y+h,:x+w], img_clothes[y:y+h,:x+w],
                                                      mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,:x+w] = cv2.add(img_clothes[y:y+h,:x+w], img_roi)
        
        # 오른쪽으로 넘어갔을 때
        elif x+w> max_x:
            img_roi = img_roi[:, :-(x+w-max_x)]
            max_x -= x
            img_clothes[y:y+h,x:x+w] = cv2.bitwise_and(img_clothes[y:y+h,x:x+w], img_clothes[y:y+h,x:x+w],
                                                       mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,x:x+w] = cv2.add(img_clothes[y:y+h,x:x+w], img_roi)
        
        # 정상적으로 내부에 들어왔을 때
        else:    
            img_clothes[y:y+h,x:x+w] = cv2.bitwise_and(img_clothes[y:y+h,x:x+w], img_clothes[y:y+h,x:x+w], 
                                                       mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,x:x+w] = cv2.add(img_clothes[y:y+h,x:x+w], img_roi)
        
    # 우클릭 뗏을 때
    elif event == cv2.EVENT_RBUTTONUP:
        img_clothes = img_before.copy()
        
    # 좌클릭 했을 때
    elif event == cv2.EVENT_LBUTTONDOWN:
        # 오른쪽 위로 넘어갔을 때
        if x+w> max_x and y < 0:
            img_roi = img_roi[(-y):, :-(x+w-max_x)]
            max_x -= x
            min_y += -y
            img_clothes[:y+h,x:x+w] = cv2.bitwise_and(img_clothes[:y+h,x:x+w], img_clothes[:y+h,x:x+w],
                                                      mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[:y+h,x:x+w] = cv2.add(img_clothes[:y+h,x:x+w], img_roi)
        
        # 오른쪽 아래로 넘어갔을 때
        elif y+h > max_y and x+w > max_x:
            img_roi = img_roi[:-(y+h-max_y), :-(x+w-max_x)]
            max_y -= y
            max_x -= x
            img_clothes[y:y+h,x:x+w] = cv2.bitwise_and(img_clothes[y:y+h,x:x+w], img_clothes[y:y+h,x:x+w], 
                                                       mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,x:x+w] = cv2.add(img_clothes[y:y+h,x:x+w], img_roi)
        
        # 왼쪽 위로 넘어갔을 때
        elif y < 0 and x < 0:
            img_roi = img_roi[(-y):, (-x):]
            min_y += -y
            min_x += -x
            img_clothes[:y+h,:x+w] = cv2.bitwise_and(img_clothes[:y+h,:x+w], img_clothes[:y+h,:x+w], 
                                                     mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[:y+h,:x+w] = cv2.add(img_clothes[:y+h,:x+w], img_roi) 
            
        # 왼쪽 아래로 넘어갔을 때
        elif x < 0 and y+h > max_y:
            img_roi = img_roi[:-(y+h-max_y), (-x):]
            max_y -= y
            min_x += -x
            img_clothes[y:y+h,:x+w] = cv2.bitwise_and(img_clothes[y:y+h,:x+w], img_clothes[y:y+h,:x+w],
                                                      mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,:x+w] = cv2.add(img_clothes[y:y+h,:x+w], img_roi)
        
        # 위로 넘어갔을 때
        elif y < 0:
            img_roi = img_roi[(-y):, :]
            min_y += -y
            img_clothes[:y+h,x:x+w] = cv2.bitwise_and(img_clothes[:y+h,x:x+w], img_clothes[:y+h,x:x+w],
                                                      mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[:y+h,x:x+w] = cv2.add(img_clothes[:y+h,x:x+w], img_roi)
          
        # 아래로 넘어갔을 때
        elif y+h > max_y:
            img_roi = img_roi[:-(y+h-max_y), :]
            max_y -= y
            img_clothes[y:y+h,x:x+w] = cv2.bitwise_and(img_clothes[y:y+h,x:x+w], img_clothes[y:y+h,x:x+w],
                                                       mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,x:x+w] = cv2.add(img_clothes[y:y+h,x:x+w], img_roi)
            
        # 왼쪽으로 넘어갔을 때
        elif x < 0:
            img_roi = img_roi[:, (-x):]
            min_x += -x
            img_clothes[y:y+h,:x+w] = cv2.bitwise_and(img_clothes[y:y+h,:x+w], img_clothes[y:y+h,:x+w],
                                                      mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,:x+w] = cv2.add(img_clothes[y:y+h,:x+w], img_roi)
        
        # 오른쪽으로 넘어갔을 때
        elif x+w> max_x:
            img_roi = img_roi[:, :-(x+w-max_x)]
            max_x -= x
            img_clothes[y:y+h,x:x+w] = cv2.bitwise_and(img_clothes[y:y+h,x:x+w], img_clothes[y:y+h,x:x+w],
                                                       mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,x:x+w] = cv2.add(img_clothes[y:y+h,x:x+w], img_roi)
        
        # 정상적으로 내부에 들어왔을 때
        else:    
            img_clothes[y:y+h,x:x+w] = cv2.bitwise_and(img_clothes[y:y+h,x:x+w], img_clothes[y:y+h,x:x+w], 
                                                       mask=not_alpha_area[min_y:max_y, min_x:max_x])
            img_clothes[y:y+h,x:x+w] = cv2.add(img_clothes[y:y+h,x:x+w], img_roi)


######################## main 부분 ########################
import cv2
import numpy as np

img = cv2.imread('img/blank_900.png', cv2.IMREAD_UNCHANGED) # 알파채널까지 그대로 읽어오기
img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
img_clothes = img.copy() 
img_before = img_clothes.copy() # 미리보기 기능 수행을 위한 임시 변수
key = 97 # 초기값 'a'의 ascii code

while True:    
    # q - 종료
    if key == 113:
        cv2.destroyAllWindows()
        break
    # s - 저장
    elif key == ord('s'):
        cv2.imwrite('img/clothes_cody.png', img_clothes)
    # i - 이미지 초기화
    elif key == ord('i'):
        img_clothes = img.copy()
    # a - 새 이미지 추가
    elif key == ord('a'):
        while True:
            img_file = input("이미지 경로 :") #'img/t4.png' #
            img_add = cv2.imread(img_file, cv2.IMREAD_UNCHANGED)
            if img_add is None:
                print("경로에 이미지가 존재하지 않습니다.")
                continue
                
            cv2.imshow('Cody', img_clothes)
            cv2.setMouseCallback('Cody', mouse_callback_cody) # 마우스 콜백 함수 등록
            h, w = img_add.shape[:2] # 가로와 세로 길이
            _, alpha_area = cv2.threshold(img_add[:,:,3], 1, 255, cv2.THRESH_BINARY) # 알파채널에 대한 이진화
            not_alpha_area = cv2.bitwise_not(alpha_area) # 배경부분 선택
            img_add = cv2.bitwise_and(img_add, img_add, mask=alpha_area) # 더할 이미지 선택
            break
            
    cv2.imshow('Cody', img_clothes)
    key = cv2.waitKey(1)
cv2.destroyAllWindows()

# 이미지 경로 예시 :  img/p3.png   img/t3.png

이미지 경로 :img/p5.png
이미지 경로 :img/t4.png
